In [ ]:
import tensorflow as tf
numGPUs = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", numGPUs)

In [ ]:
import random
import pandas
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D,Dropout,Activation,MaxPooling2D, GlobalAveragePooling2D
from keras import backend as K
import talos
import os
import math
import threading
from matplotlib import pyplot
import tensorflow_datasets as tfds
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
from ADAMApprox import AdamApprox

In [ ]:
numClients = 1

In [ ]:
num_classes = 10
dataset_name = 'svhn_cropped'
experiment_name = 'svhn'

In [ ]:
#global parameters for grid search over activation function
degree = 3
interval = 3
def reluApprox(x):
    if degree == 3:  
        if interval == 3:
            return 0.7146 + 1.5000*K.pow(x/interval,1)+0.8793*K.pow(x/interval,2)
        if interval == 5:
            return 0.7865 + 2.5000*K.pow(x/interval,1)+1.88*K.pow(x/interval,2)
        if interval == 7:
            return 0.9003 + 3.5000*K.pow(x/interval,1)+2.9013*K.pow(x/interval,2)
        if interval == 10:
            return 1.1155 + 5*K.pow(x/interval,1)+4.4003*K.pow(x/interval,2)
        if interval == 12:
            return 1.2751 + 6*K.pow(x/interval,1)+5.3803*K.pow(x/interval,2)
    if degree == 5:  
        if interval == 7:
            return 0.7521 + 3.5000*K.pow(x/interval,1)+4.3825*K.pow(x/interval,2)-1.7281*K.pow(x/interval,4)
        if interval == 20:
            return 1.3127 + 10*K.pow(x/interval,1)+15.7631*K.pow(x/interval,2)-7.6296*K.pow(x/interval,4)
def sigmoidApprox(x):
    if degree == 3:  
        if interval == 3:
            return 0.5 + 0.6997*K.pow(x/interval,1)-0.2649*K.pow(x/interval,3)
        if interval == 5:
            return 0.5 + 0.9917*K.pow(x/interval,1)-0.5592*K.pow(x/interval,3)
        if interval == 7:
            return 0.5 + 1.1511*K.pow(x/interval,1)-0.7517*K.pow(x/interval,3)
        if interval == 8:
            return 0.5 + 1.2010*K.pow(x/interval,1)-0.8156*K.pow(x/interval,2)
        if interval == 12:
            return 0.5 + 1.2384*K.pow(x/interval,1)-0.8647*K.pow(x/interval,2)
def tanApprox(x):
    if degree == 3: 
        if interval ==1:
            return 0.9797*K.pow(x/interval,1)-0.2268*K.pow(x/interval,3)        
        if interval ==2:
            return 1.7329*K.pow(x/interval,1)-0.8454*K.pow(x/interval,3)
        if interval == 3:
            return 2.1673*K.pow(x/interval,1)-1.3358*K.pow(x/interval,3)
        if interval == 5:
            return 2.5338*K.pow(x/interval,1)-1.8051*K.pow(x/interval,3)
        if interval == 7:
            return 2.6629*K.pow(x/interval,1) -1.9801*K.pow(x/interval,3)
        if interval == 12:
            return 2.7599*K.pow(x/interval,1)-2.1140*K.pow(x/interval,2)
    if degree == 12:
        print('ooopssss')

In [ ]:
# define the grid search parameters
def get_params_for_X_clients(x):    
    global interval
    
    batch_size = [16]
    epochs = [12]
    learn_rate = [0.3]
    momentum = [0.91]
    act_fn = [reluApprox]
    interval = 7
    
    param_grid = dict(learn_rate=learn_rate, momentum=momentum, batch_size=batch_size, epochs=epochs, act_fn=act_fn)
    
    return param_grid

In [ ]:
def get_model(params):
    model = Sequential()
    model.add(Conv2D(6, (5,5), activation=params['act_fn'], input_shape=input_shape))
    model.add(AveragePooling2D((2,2)))
    model.add(Conv2D(16, (5,5), activation=params['act_fn']))
    model.add(AveragePooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(120, activation=params['act_fn']))
    model.add(Dense(84, activation=params['act_fn']))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [ ]:
# Model / data parameters
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tfds.as_numpy(tfds.load(dataset_name,
                                                               split=['train','test'],
                                                               batch_size=-1,
                                                               as_supervised=True))

# Scale images to the [0, 1] range
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = x_train / 255
x_test = x_test / 255
# Make sure images have shape (28, 28, 1)
print(x_train.shape)
print(x_test.shape)
#x_train = np.expand_dims(x_train, -1)
#x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")
y_test_uncat = y_test

#to use with mean sq error
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#shuffle both train and test once (to average between runs later on..)
shuffler = np.random.permutation(len(x_train))
x_train = x_train[shuffler]
y_train = y_train[shuffler]
shuffler = np.random.permutation(len(x_test))
x_test = x_test[shuffler]
y_test = y_test[shuffler]

sample_height = x_train[0].shape[0]
sample_width = x_train[0].shape[1]
sample_channels = x_train[0].shape[2]


input_shape = (sample_height, sample_width, sample_channels)

def randomSplitClientsData(data,labels,numParties):
    numSamplesPerClient = int(data.shape[0]/numParties)
    print(numSamplesPerClient)
    clientsData = np.zeros((numParties,int(numSamplesPerClient),sample_height,sample_width,sample_channels))
    clientsDataLabels = np.zeros((numParties,int(numSamplesPerClient),num_classes))
    #print(numSamplesPerClient)
    ind = 0
    for i in range(numParties):
        clientsData[i] = data[ind:ind+numSamplesPerClient]
        clientsDataLabels[i]=labels[ind:ind+numSamplesPerClient]
        ind = ind+numSamplesPerClient
    return clientsData, clientsDataLabels

def prepare_data_for_X_clients(numClients):
    clientsData, clientsDataLabels = randomSplitClientsData(x_train, y_train, numClients)
    return clientsData, clientsDataLabels

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history, params):
    print('##########################################################')
    print(params)
    # plot loss
    pyplot.subplot(211)
    pyplot.title('MSE')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    #filename = sys.argv[0].split('/')[-1]
    #pyplot.savefig(filename + '_plot.png')
    #pyplot.close()
    pyplot.show()
    print('##########################################################')

In [ ]:
params = get_params_for_X_clients(numClients)
print(params)

In [ ]:
def test_metrics_for_X_clients(numClients, clientsData, clientsDataLabels, test_params, keep_range):

    metrics_res = np.zeros(numClients, dtype=object)
    
    
    def client_test_metrics(i):
        #Distribute load accross GPUs
        with tf.device('/GPU:'+str(i%numGPUs)):
            
            print('training in client ', i)
            model = get_model(test_params)
            
            optimizer = AdamApprox(keep_range=keep_range)
            print('Precision :', precision)
            optimizer.set_precision(precision)
            model.compile(loss="categorical_crossentropy", optimizer=optimizer, 
                          metrics=["accuracy", tf.keras.metrics.Recall(),tf.keras.metrics.Precision()],
                          run_eagerly=keep_range)
            
            hist.append(model.fit(x=clientsData[i],
                        y=clientsDataLabels[i],
                        epochs=test_params['epochs'],
                        batch_size=test_params['batch_size'],
                        verbose=1))
            
            hist_precision.append(optimizer.get_range())
            
            print('evaluation in client ', i)
            metrics = model.evaluate(x_test, y_test)
            metrics_res[i] = metrics
            
        
    # Batch multithreading
    n_batch = int(math.ceil(float(numClients)/(numGPUs)))
    print(n_batch)
    remaining_clients = numClients
    for i in range(n_batch):
        threads = list()

        for j in range(min(numGPUs, remaining_clients)):
            t = threading.Thread(target=client_test_metrics, args=(i*(numGPUs)+j,))
            threads.append(t)
            t.start()

        for _,t in enumerate(threads):
            remaining_clients -= 1
            t.join()
    
    return metrics_res

In [ ]:
hist = []
hist_precision = []
precisions = [0,0,0,0,0,0,0,0,0,0,
              1,1,1,1,1,1,1,1,1,1,
              2,2,2,2,2,2,2,2,2,2,
              4,4,4,4,4,4,4,4,4,4,
              6,6,6,6,6,6,6,6,6,6,
              8,8,8,8,8,8,8,8,8,8,
              10,10,10,10,10,10,10,10,10,10,
              12,12,12,12,12,12,12,12,12,12,
              'full','full','full','full','full''full','full','full','full','full']
test_params = dict(learn_rate = params['learn_rate'][0],
                   momentum = params['momentum'][0],
                   batch_size = params['batch_size'][0],
                   epochs = params['epochs'][0],
                   act_fn = params['act_fn'][0])

precision_res = [0] * len(precisions)
clientsData, clientsDataLabels = prepare_data_for_X_clients(numClients)

for i, p in enumerate(precisions):
    global precision
    precision = p
    keep_range = p == 'full'
    precision_res[i] = test_metrics_for_X_clients(numClients, clientsData, clientsDataLabels, test_params, keep_range)

In [ ]:
flat_list = [x for sublist in hist_precision for x in sublist]
coeff_range = [min(flat_list), max(flat_list)]
coeff_range

In [ ]:
res = np.zeros(4,dtype=float)
for i,p in enumerate(precision_res):
    res[0] += p[0][0]
    res[1] += p[0][1]
    res[2] += p[0][2]
    res[3] += p[0][3]
    if i % 10 == 9:
        print(res/10)
        print()
        res = np.zeros(4,dtype=float)